In [ ]:
%pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 72.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 96.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.8/510.8 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.4/128.4 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 95.9 MB/s eta 0

In [ ]:
# app.py
import os
import openai
import chromadb
from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv
import time
import re

In [ ]:
class Chatbot:
    """Encapsulates the OpenAI client and response generation logic with Query Expansion."""
    def __init__(self, api_key, model="gpt-4o-mini", base_url="https://api.tapsage.com/openai/v1"):
        if not api_key:
            raise ValueError("OpenAI API key is required.")
        self.model = model
        self.client = openai.OpenAI(api_key=api_key, base_url=base_url)

    def expand_query(self, query):
        """ Uses the LLM to correct potential spelling or grammatical errors in the query.
        Returns exactly two queries: [original, corrected] (if corrected is different).
        """
        if not self.client:
            print("OpenAI client not initialized. Skipping query expansion.")
            return [query, query]

        correction_prompt = f"""
        شما یک ویرایشگر زبان فارسی هستید. وظیفه شما این است که فقط و فقط اشتباهات املایی، نگارشی یا دستوری سوال زیر را اصلاح کنید.
        - اگر به صورت رسمی نوشته نشده آن را به صورت رسمی بازنویسی کنید.
        - اگر سوال از نظر زبانی صحیح است، همان سوال را بدون هیچ تغییری برگردانید.
        - هیچ بازنویسی معنادار، تغییر سبک یا جایگزینی واژه انجام ندهید.
        - فقط یک خط پاسخ بدهید و هیچ توضیح اضافه ننویسید.

        سوال: "{query}"
        """

        try:
            print("\n-> Correcting query for grammar/spelling...")
            completion = self.client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": "شما یک ویرایشگر دقیق زبان فارسی هستید."},
                    {"role": "user", "content": correction_prompt}
                ],
                temperature=0.0,  # deterministic output
            )
            corrected = completion.choices[0].message.content.strip()

            # Ensure we have a valid string
            if not corrected:
                corrected = query

            # Return exactly two queries: original + corrected (even if same)
            queries = [query, corrected]
            print(f"   Original: '{query}'")
            print(f"   Corrected: '{corrected}'")
            return queries

        except Exception as e:
            print(f"   Error during query correction: {e}")
            return [query]  # fallback: duplicate original


    def retrieve_relevant_chunks(self, queries, collection, embedding_model, n_results=10):
        """
        Retrieves the top N unique, most relevant chunks from ChromaDB for a list of queries.
        """
        all_retrieved_docs = {}

        print("-> Retrieving relevant chunks...")
        for q in queries:
            query_embedding = embedding_model.encode(q).tolist()
            results = collection.query(
                query_embeddings=[query_embedding],
                n_results=n_results
            )

            for doc in results['documents'][0]:
                if doc not in all_retrieved_docs:
                    all_retrieved_docs[doc] = True

        retrieved_texts = list(all_retrieved_docs.keys())
        print(f"   Retrieved {len(retrieved_texts)} unique chunks.")
        return retrieved_texts

    def generate_response(self, query, context_chunks):
        """
        Generates a final response using the LLM based on the query and retrieved context.
        """
        if not self.client:
            return "OpenAI client is not initialized. Cannot generate response."

        if not context_chunks:
            return "متاسفانه اطلاعات کافی برای پاسخ به سوال شما در پایگاه دانش من وجود ندارد."

        # 🔍 نمایش chunkهای بازیابی‌شده برای دیباگ/شفافیت
        print("\n--- Retrieved Knowledge Chunks (for context) ---")
        for i, chunk in enumerate(context_chunks, 1):
            print(f"\n[Chunk {i}]\n{chunk}\n{'-'*50}")
        print("------------------------------------------------\n")

        context_string = "\n\n---\n\n".join(context_chunks)

        system_prompt = """
        شما یک دستیار خدمات مشتری متخصص و دوستانه برای یک شرکت هستید. نام شما «پشتیبان» است.
        شما باید فقط و فقط بر اساس «زمینه» ارائه شده به «سوال» کاربر پاسخ دهید.
        اطلاعاتی را از خودتان اضافه نکنید. اگر زمینه برای پاسخ دادن کافی نیست،
        مودبانه بگویید که اطلاعات کافی برای پاسخ به آن سوال را ندارید.
        ***شما باید همیشه و فقط به زبان فارسی پاسخ دهید.***
        """

        user_prompt = f"زمینه: \"{context_string}\"\n\nسوال: \"{query}\""

        try:
            print("-> Generating final response...")
            start_time = time.time()
            completion = self.client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                temperature=0.5,
            )
            end_time = time.time()
            print(f"   Generated response in {end_time - start_time:.2f} seconds.")
            return completion.choices[0].message.content
        except openai.APIError as e:
            print(f"   OpenAI API Error: {e}")
            return "متاسفانه در ارتباط با هوش مصنوعی خطایی رخ داد. لطفا دوباره تلاش کنید."
        except Exception as e:
            print(f"   An unexpected error occurred: {e}")
            return "یک خطای غیرمنتظره رخ داد. لطفا با پشتیبانی تماس بگیرید."

In [ ]:
def run_chatbot():
    """
    Main function to initialize and run the command-line chatbot.
    """
    print("--- Starting Semantic Search Chatbot (CLI) ---")

    # 1. Load Environment Variables
    # load_dotenv()
    # api_key = os.environ.get("OPENAI_API_KEY")
    api_key = "tpsg-HURAqpPOuLGZtEReVzj3unfTRXGp45o"
    if not api_key:
        print("FATAL: OPENAI_API_KEY not found. Please set it in your .env file.")
        return

    # 2. Load Embedding Model
    print("Loading embedding model...")
    try:
        model_name = "paraphrase-multilingual-mpnet-base-v2"
        embedding_model = SentenceTransformer(model_name)
        print("Embedding model loaded.")
    except Exception as e:
        print(f"FATAL: Could not load embedding model. Error: {e}")
        return

    # 3. Initialize ChromaDB
    print("Initializing ChromaDB client...")
    try:
        client = chromadb.PersistentClient(path="./chroma_db")
        collection_name = "farsi_rag_collection"
        collection = client.get_collection(name=collection_name)
        print("ChromaDB client and collection loaded successfully.")
    except Exception as e:
        print(f"FATAL: Could not connect to ChromaDB. Have you run ingest.py first? Error: {e}")
        return

    # 4. Initialize Chatbot
    try:
        chatbot = Chatbot(api_key=api_key, base_url="https://api.tapsage.com/openai/v1")
        print("Chatbot client initialized.")
    except ValueError as e:
        print(f"FATAL: {e}")
        return
    print("\n--- Chatbot is ready ---")


    print("برای خروج 'خروج' را تایپ کنید")
    # 5. Start Chat Loop
    while True:
        user_query = input("شما: ")
        if user_query.lower() in ['خروج', 'exit']:
            break

        # 1. Expand the user's query
        expanded_queries = chatbot.expand_query(user_query)

        # 2. Retrieve relevant context using all queries
        context_chunks = chatbot.retrieve_relevant_chunks(expanded_queries, collection, embedding_model)

        # 3. Generate the final answer
        answer = chatbot.generate_response(user_query, context_chunks)

        print(f"\nپشتیبان: {answer}\n")

In [ ]:
run_chatbot()

--- Starting Semantic Search Chatbot (CLI) ---
Loading embedding model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding model loaded.
Initializing ChromaDB client...
ChromaDB client and collection loaded successfully.
Chatbot client initialized.

--- Chatbot is ready ---
برای خروج 'خروج' را تایپ کنید
شما: سلام

-> Correcting query for grammar/spelling...
   Original: 'سلام'
   Corrected: 'سلام'
-> Retrieving relevant chunks...
   Retrieved 10 unique chunks.

--- Retrieved Knowledge Chunks (for context) ---

[Chunk 1]
راه‌های تماس با ما به شرح زیر است:
- تلفن همراه: ۰۹۹-۸۱۸۱-۸۹۷۸
- تلفن ثابت: ۰۱۱-۴۲۴۳-۳۴۴۳
- فکس: ۰۱۱-۴۲۴۳-۳۴۴۴
- ایمیل: info@tabenergy.ir
- آدرس: مازندران، سوادکوه شمالی، شهرک صنعتی بشل، فاز ۲، همت ۷
--------------------------------------------------

[Chunk 2]
فرصتی برای همکاری پایدار با تابان انرژی: شرکت تابان انرژی مازندسو با هدف توسعه زیرساخت های انرژی پاک و گسترش استفاده از فناوری های خورشیدی در سراسر کشور، از همکاران فعال در حوزه فروش، نصب و خدمات انرژی تجدیدپذیر دعوت به همکاری می نماید. اگر شما دارای تخصص، تجربه یا شبکه فروش در زمینه تجهیزات خورشیدی هستید، می توانید به جمع 

KeyboardInterrupt: Interrupted by user